# Codebusters KPIs and Other Usefull Patterns

## import libraries

In [344]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob

Import and clean `all_tickets_2022` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all service request

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (SR)") AND resolved is not EMPTY AND resolutiondate >= 2022-12-19

In [345]:
all_sr_origin = pd.read_csv('DataSets/MainTickets/all_service_requests.csv')
all_sr_origin

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.37,Comment.38,Comment.39,Comment.40,Comment.41,Comment.42,Comment.43,Comment.44,Comment.45,Comment.46
0,Porsche - Update hidden section and dfd text,CI-9041,491359,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Porsche - please rename locations on Staging,CI-9015,490207,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MFTBC - Fuel Price Update AUGUST 2023,CI-9011,490091,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ES Market Template - Mileage rate update from ...,CI-9001,489686,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ProcurX EverC - Add FDFs and receipts,CI-8999,489633,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,Aquila Capital - Expense issue with export and...,CI-6926,393799,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466,VW - Splitting of approval for expense statements,CI-6911,393585,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,Porsche - Adjustment of approval process for n...,CI-6901,393357,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
468,Hugo Boss - add 3 new meal receipt types for t...,CI-6825,389774,Expense Delivery,Closed,CI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Select from the dataset only columns necessary for the further analysis

In [346]:
all_sr = all_sr_origin[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)']]
all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2948666729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request"},axis=1, inplace=True)


In [347]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        470 non-null    object
 1   Issue id         470 non-null    int64 
 2   Status           470 non-null    object
 3   Reporter         465 non-null    object
 4   Created          470 non-null    object
 5   Due Date         470 non-null    object
 6   Resolved         470 non-null    object
 7   Customer         470 non-null    object
 8   Type of Request  470 non-null    object
dtypes: int64(1), object(8)
memory usage: 33.2+ KB


In [348]:
all_sr.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           5
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
dtype: int64

### Convert date table to `datetime` object

In [349]:
all_sr["Created"] = pd.to_datetime(all_sr["Created"])
all_sr["Resolved"] = pd.to_datetime(all_sr["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2794271155.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_sr["Created"] = pd.to_datetime(all_sr["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2794271155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["Created"] = pd.to_datetime(all_sr["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2794271155.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_sr["Resolved"

In [350]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        470 non-null    object        
 1   Issue id         470 non-null    int64         
 2   Status           470 non-null    object        
 3   Reporter         465 non-null    object        
 4   Created          470 non-null    datetime64[ns]
 5   Due Date         470 non-null    object        
 6   Resolved         470 non-null    datetime64[ns]
 7   Customer         470 non-null    object        
 8   Type of Request  470 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 33.2+ KB


In [351]:
all_sr.shape

(470, 9)

In [352]:
all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]
all_sr

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2540177896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr["ticket_age"] = all_sr["Resolved"] - all_sr["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00
...,...,...,...,...,...,...,...,...,...,...
465,CI-6926,393799,Closed,NaN,2022-05-25 12:42:00,12/Jul/22 12:00 AM,2023-01-27 19:53:00,Aquila Capital,Service Request (SR),247 days 07:11:00
466,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00
467,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00
468,CI-6825,389774,Closed,NaN,2022-05-11 11:16:00,16/May/22 12:00 AM,2023-01-24 12:32:00,Hugo Boss,Service Request (SR),258 days 01:16:00


In [353]:
all_sr['Created'].sort_values(ascending=False)

0     2023-08-01 08:56:00
1     2023-07-26 14:29:00
2     2023-07-26 09:00:00
3     2023-07-25 04:15:00
4     2023-07-24 15:46:00
              ...        
465   2022-05-25 12:42:00
466   2022-05-24 15:11:00
467   2022-05-23 13:21:00
468   2022-05-11 11:16:00
469   2021-11-05 17:04:00
Name: Created, Length: 470, dtype: datetime64[ns]

**Summary**
- there is not SR ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

# Prepare data with individuals tasks for service requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [354]:
all_individual_tasks_path = glob.glob("DataSets/IndividualTasks/" + "*.csv")


all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\150410908.py:4: DtypeWarning: Columns (216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\150410908.py:4: DtypeWarning: Columns (25,222,223,224,225,226,227,228,229,230,264) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename).assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\150410908.py:4: DtypeWarning: Columns (27,37,38,39,78,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,329,330,331) have mixed types. Specify dtype op

,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,Verify,CI-10243141,485033,475122,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1313,Import to Customer Test System,CI-10243140,485027,475122,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1314,Communicate,CI-10243139,485025,475132,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1315,Communicate,CI-10243138,485023,482812,Individual Task,Done,CI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [355]:
all_individual_tasks_combined[['source', 'Created']].sort_values(by='Created', ascending=True)

,source,Created
3765,DataSets/IndividualTasks\from_20220901_till_20...,01/09/2022 02:07
3764,DataSets/IndividualTasks\from_20220901_till_20...,01/09/2022 03:11
3763,DataSets/IndividualTasks\from_20220901_till_20...,01/09/2022 04:22
3762,DataSets/IndividualTasks\from_20220901_till_20...,01/09/2022 09:07
3761,DataSets/IndividualTasks\from_20220901_till_20...,01/09/2022 09:29
...,...,...
1306,DataSets/IndividualTasks\from_20230331_till_20...,31/May/23 5:09 PM
1305,DataSets/IndividualTasks\from_20230331_till_20...,31/May/23 5:09 PM
1304,DataSets/IndividualTasks\from_20230331_till_20...,31/May/23 7:07 PM
1362,DataSets/IndividualTasks\from_20230331_till_20...,31/May/23 8:35 AM


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [356]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Sprint.44,Sprint.45,Sprint.46,Sprint.47,Sprint.48,Comment.2,Comment.3,Comment.4,Comment.5,Comment.6
0,0,Configure,CI-10230383,438469,437773,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Import to Test System,CI-10230381,438460,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Configure,CI-10230380,438459,437698,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Import in Production,CI-10230377,438438,432387,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import to Customer Test System,CI-10230376,438437,435618,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11301,1312,Verify,CI-10243141,485033,475122,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11302,1313,Import to Customer Test System,CI-10243140,485027,475122,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11303,1314,Communicate,CI-10243139,485025,475132,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11304,1315,Communicate,CI-10243138,485023,482812,Individual Task,Done,CI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [358]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Configure,438469,437773,30/12/2022 13:56,02/01/2023 13:18
1,Import to Test System,438460,437698,30/12/2022 13:14,02/01/2023 13:33
2,Configure,438459,437698,30/12/2022 13:14,30/12/2022 13:14
3,Import in Production,438438,432387,30/12/2022 11:38,05/01/2023 09:48
4,Import to Customer Test System,438437,435618,30/12/2022 11:01,03/01/2023 12:17
...,...,...,...,...,...
11301,Verify,485033,475122,03/Jul/23 9:14 AM,11/Jul/23 1:01 PM
11302,Import to Customer Test System,485027,475122,03/Jul/23 8:52 AM,03/Jul/23 9:14 AM
11303,Communicate,485025,475132,03/Jul/23 8:40 AM,19/Jul/23 9:47 AM
11304,Communicate,485023,482812,03/Jul/23 8:38 AM,06/Jul/23 7:01 AM


In [359]:
all_individual_tasks[all_individual_tasks['Parent id'] == 465210]

,Summary,Issue id,Parent id,Created,Resolved
7612,Communicate,477694,465210,07/Jun/23 1:56 PM,NaN
7679,Import in Production,477344,465210,06/Jun/23 3:45 PM,07/Jun/23 1:56 PM
7680,Configure,477343,465210,06/Jun/23 3:45 PM,06/Jun/23 3:45 PM
7681,Review,477342,465210,06/Jun/23 3:44 PM,06/Jun/23 3:45 PM
9160,Explain,466478,465210,20/Apr/23 8:22 AM,26/Apr/23 10:33 AM
9367,Review,465211,465210,17/Apr/23 9:43 AM,20/Apr/23 8:22 AM


### Convert date like columns to `datetime` object

In [360]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11306 entries, 0 to 11305
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    11306 non-null  object        
 1   Issue id   11306 non-null  int64         
 2   Parent id  11306 non-null  int64         
 3   Created    11306 non-null  datetime64[ns]
 4   Resolved   10860 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 441.8+ KB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [361]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                                   1983
Review                                      1912
Verify                                      1863
Communicate                                 1640
Import to Test System                       1449
Import in Production                        1283
Import to Customer Test System               705
Explain                                      254
Do                                           105
Clarify                                      104
CLONE - Review                                 1
Contact Mattia about risks assesment           1
Rename the repository                          1
Check secrets with ITS                         1
Configure2                                     1
Align with system architect and EMS team       1
Prepare a detailed plan for the next PI        1
Verify technical limitations                   1
Name: count, dtype: int64

In [362]:
all_individual_tasks.sort_values(by='Created', ascending=False).head()

,Summary,Issue id,Parent id,Created,Resolved
9989,Communicate,493537,491736,2023-08-10 22:24:00,2023-08-10 22:24:00
9990,Import in Production,493531,491736,2023-08-10 17:45:00,2023-08-10 22:24:00
9991,Configure,493530,491736,2023-08-10 17:45:00,2023-08-10 17:45:00
9992,Configure,493527,491998,2023-08-10 17:25:00,2023-08-11 10:29:00
9993,Review,493526,491736,2023-08-10 17:17:00,2023-08-10 17:45:00


### Check if there are any `null` values

In [363]:
all_individual_tasks.isnull().sum()

Summary        0
Issue id       0
Parent id      0
Created        0
Resolved     446
dtype: int64

### Drop all rows with null value

In [364]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

### Identify and drop duplicated rows

In [365]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved
11191,Import in Production,485524,484903,2023-07-04 16:06:00,2023-07-04 16:06:00
11192,Configure,485523,484903,2023-07-04 16:06:00,2023-07-04 16:06:00
11194,Configure,485511,485500,2023-07-04 15:36:00,2023-07-04 15:36:00
11196,Import in Production,485508,484449,2023-07-04 15:34:00,2023-07-04 15:34:00
11197,Configure,485505,484449,2023-07-04 15:31:00,2023-07-04 15:34:00
...,...,...,...,...,...
11297,Review,485058,485057,2023-07-03 10:00:00,2023-07-03 14:42:00
11299,Import to Test System,485050,484149,2023-07-03 09:54:00,2023-07-03 09:54:00
11300,Configure,485049,484149,2023-07-03 09:54:00,2023-07-03 09:54:00
11302,Import to Customer Test System,485027,475122,2023-07-03 08:52:00,2023-07-03 09:14:00


In [366]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates.sort_values(by='Created', ascending=False)

,Summary,Issue id,Parent id,Created,Resolved
9989,Communicate,493537,491736,2023-08-10 22:24:00,2023-08-10 22:24:00
9990,Import in Production,493531,491736,2023-08-10 17:45:00,2023-08-10 22:24:00
9991,Configure,493530,491736,2023-08-10 17:45:00,2023-08-10 17:45:00
9992,Configure,493527,491998,2023-08-10 17:25:00,2023-08-11 10:29:00
9993,Review,493526,491736,2023-08-10 17:17:00,2023-08-10 17:45:00
...,...,...,...,...,...
3761,Communicate,411631,406493,2022-09-01 09:29:00,2022-09-01 10:03:00
3762,Configure,411619,411423,2022-09-01 09:07:00,2022-09-01 16:33:00
3763,Review,411586,411585,2022-09-01 04:22:00,2022-09-01 09:36:00
3764,Import in Production,411583,402477,2022-09-01 03:11:00,2022-09-02 09:43:00


In [367]:
all_individual_tasks_no_duplicates.sort_values(by='Created', ascending=False).head()

,Summary,Issue id,Parent id,Created,Resolved
9989,Communicate,493537,491736,2023-08-10 22:24:00,2023-08-10 22:24:00
9990,Import in Production,493531,491736,2023-08-10 17:45:00,2023-08-10 22:24:00
9991,Configure,493530,491736,2023-08-10 17:45:00,2023-08-10 17:45:00
9992,Configure,493527,491998,2023-08-10 17:25:00,2023-08-11 10:29:00
9993,Review,493526,491736,2023-08-10 17:17:00,2023-08-10 17:45:00


## Summary

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [368]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\1582069957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]


### List all available statuses of the CI ticket subtask

In [369]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                         1920
Verify                            1798
Review                            1770
Communicate                       1521
Import to Test System             1422
Import in Production              1253
Import to Customer Test System     682
Explain                            241
Clarify                            100
Do                                  78
Configure2                           1
Name: count, dtype: int64

In [370]:
all_individual_tasks_time_calculation.shape

(10786, 6)

In [371]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 438429]

,Summary,Issue id,Parent id,Created,Resolved,time_in
11,Review,438430,438429,2022-12-30 10:21:00,2023-01-09 17:06:00,10 days 06:45:00
6048,Communicate,442401,438429,2023-01-18 15:14:00,2023-02-17 08:26:00,29 days 17:12:00
6060,Import in Production,442287,438429,2023-01-18 10:51:00,2023-01-18 15:14:00,0 days 04:23:00
6313,Verify,440581,438429,2023-01-10 17:43:00,2023-01-17 17:13:00,6 days 23:30:00
6315,Import to Test System,440473,438429,2023-01-10 16:32:00,2023-01-10 17:43:00,0 days 01:11:00
6351,Configure,440062,438429,2023-01-09 17:06:00,2023-01-10 16:32:00,0 days 23:26:00


In [372]:
all_individual_tasks_time_calculation[all_individual_tasks_time_calculation["Parent id"] == 456393]["time_in"].sum()

Timedelta('28 days 23:19:00')

In [373]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,11,Review,438430,438429,2022-12-30 10:21:00,2023-01-09 17:06:00,10 days 06:45:00
1,19,Review,438405,436853,2022-12-29 16:55:00,2023-01-04 09:50:00,5 days 16:55:00
2,25,Review,438383,438382,2022-12-29 14:47:00,2023-01-04 11:44:00,5 days 20:57:00
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00
...,...,...,...,...,...,...,...
1765,11181,Review,485640,484946,2023-07-05 11:38:00,2023-07-17 15:23:00,12 days 03:45:00
1766,11189,Review,485550,485549,2023-07-04 17:44:00,2023-07-06 12:42:00,1 days 18:58:00
1767,11218,Review,485404,485403,2023-07-04 12:58:00,2023-07-11 10:34:00,6 days 21:36:00
1768,11258,Review,485166,485165,2023-07-03 13:11:00,2023-07-05 16:47:00,2 days 03:36:00


In [374]:
all_sr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        470 non-null    object         
 1   Issue id         470 non-null    int64          
 2   Status           470 non-null    object         
 3   Reporter         465 non-null    object         
 4   Created          470 non-null    datetime64[ns] 
 5   Due Date         470 non-null    object         
 6   Resolved         470 non-null    datetime64[ns] 
 7   Customer         470 non-null    object         
 8   Type of Request  470 non-null    object         
 9   ticket_age       470 non-null    timedelta64[ns]
dtypes: datetime64[ns](2), int64(1), object(6), timedelta64[ns](1)
memory usage: 36.8+ KB


### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [375]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 465210]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
1253,7681,Review,477342,465210,2023-06-06 15:44:00,2023-06-06 15:45:00,0 days 00:01:00
1513,9367,Review,465211,465210,2023-04-17 09:43:00,2023-04-20 08:22:00,2 days 22:39:00


In [376]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,349721,0 days 00:02:00
2,393585,3 days 13:22:00
3,399228,52 days 06:54:00
4,402477,0 days 18:56:00
...,...,...
1570,493136,0 days 01:12:00
1571,493152,0 days 00:10:00
1572,493232,0 days 01:35:00
1573,493498,0 days 18:26:00


In [377]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


In [378]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1575 entries, 0 to 1574
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       1575 non-null   int64          
 1   time_in_review  1575 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 24.7 KB


In [379]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review
1082,465210,2 days 22:40:00


### Check on the particular ticket if the total time is summed correctly

In [380]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
349721,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
...,...,...,...,...,...,...
493136,1,1,1,1,1,1
493152,1,1,1,1,1,1
493232,2,2,2,2,2,2


In [381]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [382]:
type(all_individual_tasks_reivew_sum)

pandas.core.frame.DataFrame

In [383]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1570,1570,493136,0 days 01:12:00
1571,1571,493152,0 days 00:10:00
1572,1572,493232,0 days 01:35:00
1573,1573,493498,0 days 18:26:00


In [384]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [385]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [386]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,349721,0 days 00:02:00
2,2,393585,3 days 13:22:00
3,3,399228,52 days 06:54:00
4,4,402477,0 days 18:56:00
...,...,...,...
1570,1570,493136,0 days 01:12:00
1571,1571,493152,0 days 00:10:00
1572,1572,493232,0 days 01:35:00
1573,1573,493498,0 days 18:26:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_sr` dataset with `all_individual_tasks_reivew_sum_df`

In [387]:
all_sr_review_time = pd.merge(all_sr, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_sr_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359.0,0 days 00:01:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207.0,0 days 00:41:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091.0,1 days 00:18:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686.0,2 days 05:26:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633.0,7 days 01:19:00
...,...,...,...,...,...,...,...,...,...,...,...,...
465,CI-6926,393799,Closed,NaN,2022-05-25 12:42:00,12/Jul/22 12:00 AM,2023-01-27 19:53:00,Aquila Capital,Service Request (SR),247 days 07:11:00,NaN,NaT
466,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585.0,3 days 13:22:00
467,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,NaN,NaT
468,CI-6825,389774,Closed,NaN,2022-05-11 11:16:00,16/May/22 12:00 AM,2023-01-24 12:32:00,Hugo Boss,Service Request (SR),258 days 01:16:00,NaN,NaT


In [388]:
all_sr_review_time.isnull().sum()

Issue key           0
Issue id            0
Status              0
Reporter            5
Created             0
Due Date            0
Resolved            0
Customer            0
Type of Request     0
ticket_age          0
Parent id          13
time_in_review     13
dtype: int64

### Investigate `null` rows after merging  main SR tickets with individual subtasks

In [389]:
null_rows = all_sr_review_time[all_sr_review_time["time_in_review"].isnull()]
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 82 to 468
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        13 non-null     object         
 1   Issue id         13 non-null     int64          
 2   Status           13 non-null     object         
 3   Reporter         9 non-null      object         
 4   Created          13 non-null     datetime64[ns] 
 5   Due Date         13 non-null     object         
 6   Resolved         13 non-null     datetime64[ns] 
 7   Customer         13 non-null     object         
 8   Type of Request  13 non-null     object         
 9   ticket_age       13 non-null     timedelta64[ns]
 10  Parent id        0 non-null      float64        
 11  time_in_review   0 non-null      timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(6), timedelta64[ns](2)
memory usage: 1.3+ KB


In [390]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
82,CI-8680,474435,Closed,alejandro.ruizrobles,2023-05-25 10:38:00,02/Jun/23 12:00 AM,2023-06-14 08:36:00,LKW Walter,Service Request (SR),19 days 21:58:00,NaN,NaT
452,CI-7306,409890,Closed,ssanz,2022-08-23 15:29:00,30/Aug/22 12:00 AM,2022-12-22 14:49:00,Porsche,Service Request (SR),120 days 23:20:00,NaN,NaT
457,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,NaN,NaT
458,CI-7221,407289,Closed,askarja,2022-08-08 23:50:00,15/Aug/22 12:00 AM,2023-01-24 10:31:00,Audi,Service Request (SR),168 days 10:41:00,NaN,NaT
459,CI-7205,406532,Closed,nsanchezruiz,2022-08-04 11:54:00,02/May/23 12:00 AM,2023-04-10 08:47:00,Hugo Boss,Service Request (SR),248 days 20:53:00,NaN,NaT
460,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,NaN,NaT
461,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,NaN,NaT
462,CI-7167,404691,Closed,askarja,2022-07-25 13:03:00,01/Aug/22 12:00 AM,2023-01-24 10:34:00,Rheinmetall,Service Request (SR),182 days 21:31:00,NaN,NaT
463,CI-7156,403721,Closed,askarja,2022-07-19 18:23:00,26/Jul/22 12:00 AM,2023-01-27 20:00:00,Traton SE,Service Request (SR),192 days 01:37:00,NaN,NaT
464,CI-7029,398756,Closed,askarja,2022-06-23 14:32:00,30/Jun/22 12:00 AM,2023-01-27 19:55:00,TeamBank AG,Service Request (SR),218 days 05:23:00,NaN,NaT


In [391]:
all_sr_review_time_without_null = all_sr_review_time.dropna()
all_sr_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

# TO DO
- update the lates dateset

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [392]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [393]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
349721,3,3,3,3,3
356993,3,3,3,3,3
359781,3,3,3,3,3
360791,1,1,1,1,1
...,...,...,...,...,...
493136,6,6,6,6,6
493152,1,1,1,1,1
493232,6,6,6,6,6


In [394]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [395]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,349721,65 days 00:18:00
2,356993,18 days 04:09:00
3,359781,29 days 18:25:00
4,360791,0 days 00:01:00
...,...,...
1704,493136,1 days 02:26:00
1705,493152,0 days 00:10:00
1706,493232,0 days 17:52:00
1707,493498,0 days 18:26:00


In [396]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [397]:
all_sr_codebusters_time = pd.merge(all_sr, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359,0 days 17:47:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207,0 days 18:31:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091,1 days 06:26:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686,8 days 14:56:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633,8 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...
456,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,6 days 05:40:00
457,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,1 days 19:49:00
458,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,8 days 18:55:00
459,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:00:00


### Investigate `null` rows after merging  main SR tickets with individual subtasks

In [398]:
all_sr_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               3
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [399]:
all_sr_codebusters_time_without_null = all_sr_codebusters_time.dropna()
all_sr_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [400]:
all_sr_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_on_codebusters
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359,0 days 17:47:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207,0 days 18:31:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091,1 days 06:26:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686,8 days 14:56:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633,8 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...
453,CI-7235,408302,Closed,askarja,2022-08-13 13:35:00,06/Sep/22 12:00 AM,2023-01-24 10:30:00,Rheinmetall,Service Request (SR),163 days 20:55:00,408302,6 days 20:20:00
454,CI-7229,407702,Closed,ssanz,2022-08-10 12:43:00,17/Aug/22 12:00 AM,2023-01-17 15:55:00,Vaillant,Service Request (SR),160 days 03:12:00,407702,42 days 11:31:00
455,CI-7227,407574,Closed,askarja,2022-08-10 00:39:00,30/Sep/22 12:00 AM,2023-01-24 11:52:00,Rheinmetall,Service Request (SR),167 days 11:13:00,407574,102 days 23:41:00
457,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,1 days 19:49:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all SR tickets dataset

In [401]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
7,Communicate,438434,427628,2022-12-30 10:49:00,2023-01-02 11:04:00,3 days 00:15:00
12,Verify,438428,421362,2022-12-30 09:59:00,2022-12-30 10:52:00,0 days 00:53:00
13,Verify,438427,435618,2022-12-30 09:58:00,2022-12-30 11:01:00,0 days 01:03:00
15,Verify,438421,436629,2022-12-30 09:13:00,2023-01-04 12:56:00,5 days 03:43:00
16,Verify,438420,434536,2022-12-30 09:05:00,2023-01-05 14:53:00,6 days 05:48:00
...,...,...,...,...,...,...
11287,Clarify,485115,471820,2023-07-03 11:34:00,2023-07-20 08:50:00,16 days 21:16:00
11298,Communicate,485051,484149,2023-07-03 09:54:00,2023-07-07 08:45:00,3 days 22:51:00
11301,Verify,485033,475122,2023-07-03 09:14:00,2023-07-11 13:01:00,8 days 03:47:00
11303,Communicate,485025,475132,2023-07-03 08:40:00,2023-07-19 09:47:00,16 days 01:07:00


In [402]:
all_individual_others_time[all_individual_others_time["Parent id"] == 456393]

,Summary,Issue id,Parent id,Created,Resolved,time_in
9462,Communicate,464825,456393,2023-04-13 14:21:00,2023-04-13 14:22:00,0 days 00:01:00
9946,Verify,462158,456393,2023-03-31 14:42:00,2023-04-03 11:08:00,2 days 20:26:00


Summarize time for each subtask outside codeubsters

In [403]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,308144,0 days 00:01:00
1,331914,0 days 00:00:00
2,335272,0 days 00:00:00
3,336670,3 days 15:01:00
4,338034,17 days 05:13:00
...,...,...
1670,492830,0 days 21:55:00
1671,492935,0 days 00:00:00
1672,493136,0 days 01:19:00
1673,493152,0 days 00:00:00


In [404]:
all_individual_others_time_sum_df[all_individual_others_time_sum_df["Parent id"] == 456393]

,Parent id,time_outside_codebusters
1072,456393,2 days 20:27:00


In [405]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_sr` with `all_individual_others_time_sum_df` dataset

In [406]:
all_sr_outside_codebusters_time = pd.merge(all_sr, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_sr_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359,0 days 00:00:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207,4 days 16:14:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091,5 days 12:29:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686,1 days 07:48:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633,0 days 05:44:00
...,...,...,...,...,...,...,...,...,...,...,...,...
454,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,405159,0 days 00:01:00
455,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00
456,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00
457,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00


In [407]:
all_sr_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_sr_outside_codebusters_time`, `all_sr_codebusters_time` and `all_sr_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [408]:
all_sr_time_on_and_outside_codebusters = pd.merge(all_sr_outside_codebusters_time, all_sr_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359,0 days 00:00:00,0 days 17:47:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207,4 days 16:14:00,0 days 18:31:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091,5 days 12:29:00,1 days 06:26:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686,1 days 07:48:00,8 days 14:56:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633,0 days 05:44:00,8 days 18:07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,126 days 07:06:00,6 days 05:40:00
452,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00,1 days 19:49:00
453,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00,8 days 18:55:00
454,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00,0 days 00:00:00


In [409]:
all_sr_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_sr_review_time` dataset

In [410]:
all_sr_with_time = pd.merge(all_sr_time_on_and_outside_codebusters, all_sr_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_sr_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),0 days 17:47:00,491359,0 days 00:00:00,0 days 17:47:00,0 days 00:01:00
1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),5 days 10:45:00,490207,4 days 16:14:00,0 days 18:31:00,0 days 00:41:00
2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),6 days 18:55:00,490091,5 days 12:29:00,1 days 06:26:00,1 days 00:18:00
3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),9 days 22:44:00,489686,1 days 07:48:00,8 days 14:56:00,2 days 05:26:00
4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),8 days 23:51:00,489633,0 days 05:44:00,8 days 18:07:00,7 days 01:19:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,407535,126 days 07:06:00,6 days 05:40:00,NaT
452,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,405141,103 days 14:07:00,1 days 19:49:00,NaT
453,CI-6911,393585,Closed,NaN,2022-05-24 15:11:00,11/Aug/22 12:00 AM,2023-01-24 10:33:00,VW,Service Request (SR),244 days 19:22:00,393585,117 days 05:54:00,8 days 18:55:00,3 days 13:22:00
454,CI-6901,393357,Closed,NaN,2022-05-23 13:21:00,30/May/22 12:00 AM,2023-03-21 13:06:00,Porsche,Service Request (SR),301 days 23:45:00,393357,0 days 00:01:00,0 days 00:00:00,NaT


Check for null values

In [411]:
all_sr_with_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    3
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              4
dtype: int64

In [412]:
all_sr_with_time.shape

(456, 14)

### Drop rows which contain null value

In [413]:
all_sr_with_time_without_null = all_sr_with_time.dropna()
all_sr_with_time_without_null.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
time_in_review              0
dtype: int64

In [414]:
all_sr_with_time_without_null.shape

(451, 14)

In [415]:
# all_sr_with_time_without_null.to_csv('all_sr.csv')

## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [416]:
all_sr_with_time_converted = all_sr_with_time_without_null
all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 451 entries, 0 to 455
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   Issue key                 451 non-null    object         
 1   Issue id                  451 non-null    int64          
 2   Status                    451 non-null    object         
 3   Reporter                  451 non-null    object         
 4   Created                   451 non-null    datetime64[ns] 
 5   Due Date                  451 non-null    object         
 6   Resolved                  451 non-null    datetime64[ns] 
 7   Customer                  451 non-null    object         
 8   Type of Request           451 non-null    object         
 9   ticket_age                451 non-null    timedelta64[ns]
 10  Parent id                 451 non-null    int64          
 11  time_outside_codebusters  451 non-null    timedelta64[ns]
 12  time_on_codeb

Convert `time in..` columns to minutes

In [417]:
all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['time_outside_codebusters'] = all_sr_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_sr_with_time_converted['ticket_age'] = all_sr_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_sr_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 451 entries, 0 to 455
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Issue key                 451 non-null    object        
 1   Issue id                  451 non-null    int64         
 2   Status                    451 non-null    object        
 3   Reporter                  451 non-null    object        
 4   Created                   451 non-null    datetime64[ns]
 5   Due Date                  451 non-null    object        
 6   Resolved                  451 non-null    datetime64[ns]
 7   Customer                  451 non-null    object        
 8   Type of Request           451 non-null    object        
 9   ticket_age                451 non-null    float64       
 10  Parent id                 451 non-null    int64         
 11  time_outside_codebusters  451 non-null    float64       
 12  time_on_codebusters       4

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\497592799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_in_review'] = all_sr_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_30708\497592799.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sr_with_time_converted['time_on_codebusters'] = all_sr_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local

In [418]:
all_sr_with_time_converted.reset_index(inplace=True)
all_sr_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,0,CI-9041,491359,Closed,yutaka.yasumi,2023-08-01 08:56:00,04/Aug/23 12:00 AM,2023-08-02 02:43:00,Porsche,Service Request (SR),1067.0,491359,0.0,1067.0,1.0
1,1,CI-9015,490207,Closed,askarja,2023-07-26 14:29:00,02/Aug/23 12:00 AM,2023-08-01 01:14:00,Porsche,Service Request (SR),7845.0,490207,6734.0,1111.0,41.0
2,2,CI-9011,490091,Closed,yutaka.yasumi,2023-07-26 09:00:00,31/Jul/23 12:00 AM,2023-08-02 03:55:00,MFTBC,Service Request (SR),9775.0,490091,7949.0,1826.0,1458.0
3,3,CI-9001,489686,Closed,ritsuko.shimizu,2023-07-25 04:15:00,28/Jul/23 12:00 AM,2023-08-04 02:59:00,Product Management,Service Request (SR),14324.0,489686,1908.0,12416.0,3206.0
4,4,CI-8999,489633,Closed,alejandro.ruizrobles,2023-07-24 15:46:00,31/Jul/23 12:00 AM,2023-08-02 15:37:00,ProcurX,Service Request (SR),12951.0,489633,344.0,12607.0,10159.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,447,CI-7253,408597,Closed,ssanz,2022-08-16 14:54:00,09/Dec/22 12:00 AM,2023-07-25 15:19:00,OBB,Service Request (SR),493945.0,408597,209053.0,192777.0,136644.0
447,448,CI-7235,408302,Closed,askarja,2022-08-13 13:35:00,06/Sep/22 12:00 AM,2023-01-24 10:30:00,Rheinmetall,Service Request (SR),235975.0,408302,198864.0,9860.0,4.0
448,449,CI-7229,407702,Closed,ssanz,2022-08-10 12:43:00,17/Aug/22 12:00 AM,2023-01-17 15:55:00,Vaillant,Service Request (SR),230592.0,407702,110233.0,61171.0,22420.0
449,450,CI-7227,407574,Closed,askarja,2022-08-10 00:39:00,30/Sep/22 12:00 AM,2023-01-24 11:52:00,Rheinmetall,Service Request (SR),241153.0,407574,1.0,148301.0,148301.0


In [419]:
all_sr_with_time_converted[all_sr_with_time_converted["Issue id"] == 456393]

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
200,200,CI-8315,456393,Closed,jbonnavion,2023-03-15 15:03:00,22/Mar/23 12:00 AM,2023-04-13 14:22:00,DERPART,Service Request (SR),41719.0,456393,4107.0,37612.0,21685.0


In [420]:
all_sr_with_time_converted['Created'].sort_values(ascending=False)

0     2023-08-01 08:56:00
1     2023-07-26 14:29:00
2     2023-07-26 09:00:00
3     2023-07-25 04:15:00
4     2023-07-24 15:46:00
              ...        
446   2022-08-16 14:54:00
447   2022-08-13 13:35:00
448   2022-08-10 12:43:00
449   2022-08-10 00:39:00
450   2021-11-05 17:04:00
Name: Created, Length: 451, dtype: datetime64[ns]

In [421]:
file_path = Path('DataSets/KPIs/all_sr_kpi.csv')
all_sr_with_time_converted.to_csv(path_or_buf=file_path)

# Investigation starts

In [422]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 438360]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
3,38,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
4,40,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00


In [423]:
all_sr[all_sr["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age
350,CI-7938,438360,Closed,nsanchezruiz,2022-12-29 11:30:00,30/Dec/22 12:00 AM,2022-12-29 15:44:00,Ineos,Service Request (SR),0 days 04:14:00


In [424]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,ticket_age,Parent id,time_in_review
82,CI-8680,474435,Closed,alejandro.ruizrobles,2023-05-25 10:38:00,02/Jun/23 12:00 AM,2023-06-14 08:36:00,LKW Walter,Service Request (SR),19 days 21:58:00,NaN,NaT
452,CI-7306,409890,Closed,ssanz,2022-08-23 15:29:00,30/Aug/22 12:00 AM,2022-12-22 14:49:00,Porsche,Service Request (SR),120 days 23:20:00,NaN,NaT
457,CI-7226,407535,Closed,NaN,2022-08-09 15:05:00,16/Aug/22 12:00 AM,2023-01-18 06:29:00,Porsche,Service Request (SR),161 days 15:24:00,NaN,NaT
458,CI-7221,407289,Closed,askarja,2022-08-08 23:50:00,15/Aug/22 12:00 AM,2023-01-24 10:31:00,Audi,Service Request (SR),168 days 10:41:00,NaN,NaT
459,CI-7205,406532,Closed,nsanchezruiz,2022-08-04 11:54:00,02/May/23 12:00 AM,2023-04-10 08:47:00,Hugo Boss,Service Request (SR),248 days 20:53:00,NaN,NaT
460,CI-7179,405159,Closed,askarja,2022-07-27 19:06:00,04/Aug/22 12:00 AM,2022-12-29 10:43:00,Eberspacher,Service Request (SR),154 days 15:37:00,NaN,NaT
461,CI-7178,405141,Closed,askarja,2022-07-27 16:18:00,03/Aug/22 12:00 AM,2023-01-24 10:32:00,msg systems,Service Request (SR),180 days 18:14:00,NaN,NaT
462,CI-7167,404691,Closed,askarja,2022-07-25 13:03:00,01/Aug/22 12:00 AM,2023-01-24 10:34:00,Rheinmetall,Service Request (SR),182 days 21:31:00,NaN,NaT
463,CI-7156,403721,Closed,askarja,2022-07-19 18:23:00,26/Jul/22 12:00 AM,2023-01-27 20:00:00,Traton SE,Service Request (SR),192 days 01:37:00,NaN,NaT
464,CI-7029,398756,Closed,askarja,2022-06-23 14:32:00,30/Jun/22 12:00 AM,2023-01-27 19:55:00,TeamBank AG,Service Request (SR),218 days 05:23:00,NaN,NaT


# Investigation ends

# To Do